# Recurrent VAE

based on implementation: https://github.com/tejaslodaya/timeseries-clustering-vae

based on paper: https://arxiv.org/pdf/1412.6581.pdf


In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch import distributions
from base import BaseEstimator
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorboardX import SummaryWriter
from seglearn.transform import InterpLongToWide, SegmentX, FeatureRep, PadTrunc
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [3]:
class Vrae(nn.Module):
    """VAE + LSTM"""
    def __init__(self, num_features, hidden_size, hidden_layer_depth, latent_length, sequence_length, output_size, batch_size, dropout):
        super(Vrae, self).__init__()
        # Parameters
        self.number_of_features = num_features
        self.hidden_size = hidden_size
        self.hidden_layer_depth = hidden_layer_depth
        self.latent_length = latent_length
        self.sequence_length = sequence_length
        self.output_size = output_size
        self.batch_size = batch_size
        
        # Encoder + reparameterization
        self.encoder = nn.LSTM(self.number_of_features, self.hidden_size, self.hidden_layer_depth, dropout=dropout)
        self.hidden_to_mean = nn.Linear(self.hidden_size, self.latent_length)
        self.hidden_to_logvar = nn.Linear(self.hidden_size, self.latent_length)
            # Initialization
        nn.init.xavier_uniform_(self.hidden_to_mean.weight)
        nn.init.xavier_uniform_(self.hidden_to_logvar.weight)
        
        # Decoder
        self.decoder = nn.LSTM(1, self.hidden_size, self.hidden_layer_depth)
        self.decoder_inputs = torch.zeros(self.sequence_length, self.batch_size, 1, requires_grad=True).type(torch.cuda.FloatTensor)
        self.c_0 = torch.zeros(self.hidden_layer_depth, self.batch_size, self.hidden_size, requires_grad=True).type(torch.cuda.FloatTensor)
        self.latent_to_hidden = nn.Linear(self.latent_length, self.hidden_size)
        self.hidden_to_output = nn.Linear(self.hidden_size, self.output_size)
            # Initialization
        nn.init.xavier_uniform_(self.latent_to_hidden.weight)
        nn.init.xavier_uniform_(self.hidden_to_output.weight)
    
    def get_latent(self, cell_output, training=True):
        self.latent_mean = self.hidden_to_mean(cell_output)
        self.latent_logvar = self.hidden_to_logvar(cell_output)
        
        if self.training:
            std = torch.exp(0.5 * self.latent_logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(self.latent_mean)
        else:
            return self.latent_mean
        
    
    def forward(self, x):
        # Encoding
        _, (h_end, c_end) = self.encoder(x)
        h_end = h_end[-1, :, :]
        # Reparameterization
        latent = self.get_latent(h_end)
        # Decoding
        h_state = self.latent_to_hidden(latent)
            # repeat vector
        h_0 = torch.stack([h_state for _ in range(self.hidden_layer_depth)])
        decoder_output, _ = self.decoder(self.decoder_inputs, (h_0, self.c_0))
        out = self.hidden_to_output(decoder_output)
        return out, latent   

In [4]:
# Parameters
hidden_size = 90
hidden_layer_depth = 1
latent_length = 20
batch_size = 32
learning_rate = 0.0005
# n_epochs = 150
dropout_rate = 0.2
num_features = 3
output_size = 9
sequence_length = 200

In [5]:
exercises = {
    0: 'PEN',
    1: 'FLEX',
    2: 'SCAP',
    3: 'ABD',
    4: 'IR',
    5: 'ER',
    6: 'DIAG',
    7: 'ROW',
    8: 'SLR'
}

In [6]:
# Model
model = Vrae(num_features, hidden_size, hidden_layer_depth, latent_length, sequence_length, output_size, batch_size, dropout_rate)
model = model.cuda()
criterion = nn.MSELoss()
criterion.size_average = False
criterion = criterion.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/home/lja/Documents/waterloo/4A/CS/assignments/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [7]:
# Dataloading
class ExerciseDataset(Dataset):
    """Shoulder exercise dataset"""
    
    def __init__(self, npy_file, length=None, transform=None, sanity_check=None):
        self.dataset = np.load(npy_file, allow_pickle=True).item()
        self.seq_length = length
        self.data = self.process_dataset(length)
        self.original_data = self.data.copy()
        self.data = self.data.astype(np.float)
        self.targets = self.dataset['exnum']
        self.targets = self.targets.reshape((self.targets.shape[0],))
        self.original_targets = self.targets.copy()
#         self.subject = self.dataset['subject']
#         self.original_subject = self.subject.copy()
        
        self.transform = transform
        
        if sanity_check is not None:
            self.data = [self.data[sanity_check]]
            self.targets = [self.targets[sanity_check]]

        assert (len(self.data) == len(self.targets))
        
    def process_dataset(self, length):
        shape = [data.shape[0] for data in self.dataset['X']]
        if length is None:
            average_len = round(sum(shape) / len(shape))
            self.seq_length = average_len
        processed, _, _ = PadTrunc(width=self.seq_length).transform(X=self.dataset['X'])
        return processed
    
    def fold(self, fold_indices):
        # Create fold for K-fold validation
        self.data = self.original_data[fold_indices]
        self.targets = self.original_targets[fold_indices]
#         self.subject = self.original_subject[fold_indices]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        return torch.from_numpy(self.data[idx][:, 1:]), self.targets[idx]

In [8]:
dataset = ExerciseDataset("cropped_resampled_acc_nar.npy", length=200)

In [9]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
assert train_size + test_size == len(dataset)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [10]:
WORKERS=6
train_loader = DataLoader(train_dataset, batch_size=32, num_workers=WORKERS, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=4, num_workers=WORKERS, shuffle=False, drop_last=True)

In [11]:
def loss_function(x_decoded, x, mean, logvar, criterion):
    kl_loss = -0.5 * torch.mean(1 + logvar - mean.pow(2) - logvar.exp())
    recon_loss = criterion(x_decoded, x)
    return kl_loss + recon_loss, recon_loss, kl_loss

In [12]:
def training(train_loader, net, optimizer, criterion, loss_graph, acc_graph):
    
    epoch_loss = 0
    epoch_acc = 0
    net.train()
#     denom = len(train_loader.dataset) // train_loader.batch_size
    
    for i, batch in enumerate(train_loader):
        
        optimizer.zero_grad()
        x_batch, y_batch = batch
        x_batch = x_batch.permute(1, 0, 2)
        x_batch = x_batch.float().cuda()
        y_batch = y_batch.long().cuda()
        
        out, latent = model(x_batch)
        print(out.shape, latent.shape)
        
        # Accuracy
        preds = F.softmax(out, dim=1).argmax(dim=1)
        correct = (preds == y_batch)
        acc = correct.sum().item() / len(correct)
        acc_graph.append(100*acc)
        
        # Loss
        loss = criterion(out, y_batch)
        loss_graph.append(loss.item())
        loss.backward()
        optimizer.step()
#         sched.step()

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)
#     return epoch_loss / denom, epoch_acc / denom


def evaluate(val_loader, net, criterion, loss_graph, acc_graph):
    
    epoch_loss = 0
    epoch_acc = 0
    net.eval()
    
#     denom = len(val_loader.dataset) // val_loader.batch_size
    
    with torch.no_grad():
        for i, val in enumerate(val_loader):
            x_val, y_val = val
            x_val = x_val.float().cuda()
            y_val = y_val.long().cuda()
    
            out, latent = model(x_val)
        
            # Accuracy
            preds = F.softmax(out, dim=1).argmax(dim=1)
            correct = (preds == y_val)
            acc = correct.sum().item() / len(correct)
            acc_graph.append(100*acc)
            
            # Loss
            loss = criterion(out, y_val)
            loss_graph.append(loss.item())
              
            epoch_loss += loss.item()
            epoch_acc += acc

    return epoch_loss / len(val_loader), epoch_acc / len(val_loader)
#     return epoch_loss / denom, epoch_acc / denom

In [13]:
%time
%matplotlib notebook

N_EPOCHS = 40

loss_graph_train = []
avg_loss_train = []
acc_graph_train = []
avg_acc_train = []
loss_graph_val = []
avg_loss_val = []
acc_graph_val = []
avg_acc_val = []

fig = plt.figure(figsize=(12,6))
fig.tight_layout()

ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = training(train_loader, model, optimizer, criterion, loss_graph_train, acc_graph_train)  
    val_loss, val_acc = evaluate(test_loader, model, criterion, loss_graph_val, acc_graph_val)
    
    # Saving best model and early stopping
    if val_acc > best_acc:
        trials = 0
        best_acc = val_acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            trials = 0
#             break
       
    avg_loss_train.append((len(loss_graph_train), train_loss))
    avg_acc_train.append((len(acc_graph_train), train_acc*100.))
    
    avg_loss_val.append((len(loss_graph_val), val_loss))
    avg_acc_val.append((len(acc_graph_val), val_acc*100.))
    
    # TRAINING
    ax1.clear()
    ax1.set_xlabel('iterations')
    ax1.set_ylabel('training loss')
    ax1.plot(loss_graph_train, label='train loss')
    ax1.plot(*zip(*avg_loss_train), label="average loss", color='red')
    ax1.legend(loc='upper left')
    
    ax2.set_ylabel('training accuracy')
    ax2.set_xlabel('iterations')
    ax2.plot(acc_graph_train, label='train acc', color='red')
    ax2.plot(*zip(*avg_acc_train), label="average accuracy", color='blue')
    
    # EVALUATION
    ax3.clear()
    ax3.set_xlabel('iterations')
    ax3.set_ylabel('validation loss')
    ax3.plot(loss_graph_val, label='validation loss')
    ax3.plot(*zip(*avg_loss_val), label="average loss", color='red')
    ax3.legend(loc='upper left') 

    ax4.set_ylabel('validation accuracy')
    ax4.set_xlabel('iterations')
    ax4.plot(acc_graph_val, label='val acc', color='red')
    ax4.plot(*zip(*avg_acc_val), label="average accuracy", color='blue')
    fig.canvas.draw()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


<IPython.core.display.Javascript object>

torch.Size([200, 32, 9]) torch.Size([32, 20])


RuntimeError: The size of tensor a (9) must match the size of tensor b (32) at non-singleton dimension 1